In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:

# Import necessary libraries
import tensorflow as tf
import scipy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'Wound_dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model creation and fine-tuning
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    verbose=1
)

# Save the model for deployment
model.save('wound_classifier.h5')


Found 431 images belonging to 7 classes.
Epoch 1/10
14/14 [==============================] - 1159s 72s/step - loss: 1.6277 - accuracy: 0.4664
Epoch 2/10
14/14 [==============================] - 636s 33s/step - loss: 0.6927 - accuracy: 0.7958
Epoch 3/10
14/14 [==============================] - 327s 22s/step - loss: 0.6211 - accuracy: 0.8005
Epoch 4/10
14/14 [==============================] - 156s 8s/step - loss: 0.6024 - accuracy: 0.8329
Epoch 5/10
14/14 [==============================] - 111s 8s/step - loss: 0.4394 - accuracy: 0.8747
Epoch 6/10
14/14 [==============================] - 162s 12s/step - loss: 0.3977 - accuracy: 0.8979
Epoch 7/10
14/14 [==============================] - 917s 58s/step - loss: 0.3646 - accuracy: 0.8933
Epoch 8/10
14/14 [==============================] - 271s 19s/step - loss: 0.3454 - accuracy: 0.8979
Epoch 9/10
14/14 [==============================] - 335s 17s/step - loss: 0.4870 - accuracy: 0.8631
Epoch 10/10
14/14 [==============================] - 435s 19

c:\Users\YASHINI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image

# Load the pre-trained model
model = tf.keras.models.load_model('wound_classifier.h5')

# Define the categories
categories = ['Abrasions', 'Bruises', 'Burns', 'Cut', 'Ingrown Nails', 'Laceration', 'Stab Wound']

# Function to predict the type of wound
def predict_wound_type(image_path):
    # Load and preprocess the user input image
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Make predictions
    predictions = model.predict(img)

    # Get the predicted class index
    predicted_class_index = np.argmax(predictions)

    # Get the predicted class label
    predicted_class_label = categories[predicted_class_index]

    return predicted_class_label

# Example usage:
user_image_path = 'sample input.jpg'
predicted_wound_type = predict_wound_type(user_image_path)
print(f'Predicted Wound Type: {predicted_wound_type}')


1/1 [==============================] - 21s 21s/step
Predicted Wound Type: Bruises
